In [1]:
import os
import sys
os.chdir('/home/csy/work/3D/PIDiff')

sys.path.append("/home/csy/work/3D/PIDiff")
sys.path.append("/home/csy/work/3D/PIDiff/utils")
sys.path.append("/home/csy/work/3D")
import argparse
import multiprocessing as mp
import pickle
import shutil
from functools import partial
from tqdm.auto import tqdm
from utils.data import PDBProtein, parse_sdf_file
from mol_tree import *
import rdkit
import rdkit.Chem as Chem
import rdkit.Chem.AllChem as AllChem
from openbabel import pybel
from openbabel import openbabel as ob
import subprocess
import os
import numpy as np
import shutil
from plip.structure.preparation import PDBComplex
import traceback
import utils.misc as misc
import utils.transforms as trans
from torch_geometric.transforms import Compose
from models.molopt_score_model import ScorePosNet3D, log_sample_categorical
from scripts.sample_diffusion import sample_diffusion_ligand
#from pl_data import ProteinLigandData, torchify_dict

In [2]:
def sdf2pdb(sdf_path, pocket_path):
    output_path = sdf_path.replace('.sdf', '.pdb')
    run_line = f'obabel -isdf {sdf_path} -opdb -O {output_path}'
    result = subprocess.run(run_line.split(), capture_output=True, check=True, universal_newlines=True)
    print(result)
    complex_path = pocket_path.replace('.pdb', 'complex.pdb')
    shutil.copy(pocket_path, complex_path)
    
    comp = open(complex_path, 'a')
    comp.write("TER\n")
    
    lig_pdb = open(output_path)
    lines = lig_pdb.readlines()
    for line in lines:
        if line[0:6] == 'HETATM' or line[0:6] == 'CONECT':
            comp.write(line)
    #comp.write("END")
    lig_pdb.close()        
    comp.close()
    return complex_path

def load_item(sdf_path, pdb_path):

    with open(pdb_path, 'r') as f:
        pdb_block = f.read()
    with open(sdf_path, 'r') as f:
        sdf_block = f.read()
    return pdb_block, sdf_block

def process_item(sdf_path, pdb_path, pocket_path):
    pdb_block, sdf_block = load_item(sdf_path, pdb_path)
    protein = PDBProtein(pdb_block)
    # ligand = parse_sdf_block(sdf_block)
    ligand = parse_sdf_file(sdf_path)

    pdb_block_pocket = protein.residues_to_pdb_block(
        protein.query_residues_ligand(ligand, 20)
    ).replace("END\n", "")    
    pocket_path = pocket_path + pdb_path.split('/')[-1].replace('.pdb', '_pocket.pdb')
    with open(pocket_path, 'w') as f:
        f.write(pdb_block_pocket)
        
    return pocket_path
        
    

조건
- pdb id에 해당하는 directory가 있어야 함 
- 해당 폴더 안에는 protein으로만 구성되어 있는 .pdb파일(직접 정제) 
- ligand에 해당하는 .sdf파일이 있어야 함. 

In [4]:
protein_name = '2IVU_RET'

sdf_path = f'/home/csy/work/3D/PIDiff/scripts/real_world_validation/Kinase/{protein_name}/ligand.sdf'
pdb_path = f'/home/csy/work/3D/PIDiff/scripts/real_world_validation/Kinase/{protein_name}/receptor.pdb'

In [5]:
# # mol2 to sdf
# mol2_path = f"/home/csy/work/3D/PIDiff/scripts/real_world_validation/{protein_name}/ligand.mol2"
# o_path = f"/home/csy/work/3D/PIDiff/scripts/real_world_validation/{protein_name}/ligand.sdf"
# run_line = f'obabel -imol2 {mol2_path} -osdf -O {o_path}'
# result = subprocess.run(run_line.split(), capture_output=True, check=True, universal_newlines=True)
# print(result)

In [5]:
pocket_path = f'/home/csy/work/3D/PIDiff/scripts/real_world_validation/Kinase/{protein_name}/'
pocket_path = process_item(sdf_path, pdb_path, pocket_path)
complex_path = sdf2pdb(sdf_path, pocket_path) 
pocket_path = f'/home/csy/work/3D/PIDiff/scripts/real_world_validation/Kinase/{protein_name}/receptor_pocket.pdb'
complex_path = f'/home/csy/work/3D/PIDiff/scripts/real_world_validation/Kinase/{protein_name}/receptor_pocketcomplex.pdb'

CompletedProcess(args=['obabel', '-isdf', '/home/csy/work/3D/targetdiff_PINN/scripts/real_world_validation/Kinase/2IVU_RET/ligand.sdf', '-opdb', '-O', '/home/csy/work/3D/targetdiff_PINN/scripts/real_world_validation/Kinase/2IVU_RET/ligand.pdb'], returncode=0, stdout='', stderr='1 molecule converted\n')


In [8]:
"""Calculate Bounding Box"""

# pocket_path = f'/home/csy/work/3D/PharDiff/scripts/real_world_validation/{protein_name}/receptor_pocket.pdb'
# complex_path = f'/home/csy/work/3D/PharDiff/scripts/real_world_validation/{protein_name}/receptor_pocketcomplex.pdb'
# ligand_path = f'/home/csy/work/3D/PharDiff/scripts/real_world_validation/{protein_name}/ligand.pdb'
# with open(pocket_path, 'r') as f:
#     lines = [l for l in f.readlines() if (l.startswith('ATOM'))]
#     pocket_x = [float(l[30:38]) for l in lines]
#     pocket_y = [float(l[38:46]) for l in lines]
#     pocket_z = [float(l[46:54]) for l in lines]    
    
# with open(ligand_path, 'r') as f:
#     lines = [l for l in f.readlines() if (l.startswith('HETATM'))]
#     ligand_x = [float(l[30:38]) for l in lines]
#     ligand_y = [float(l[38:46]) for l in lines]
#     ligand_z = [float(l[46:54]) for l in lines]  
    
# ligand_center = np.array([(max(ligand_x) + max(ligand_x)) / 2 , (max(ligand_y) + max(ligand_y)) / 2 , (max(ligand_z) + max(ligand_z)) / 2])
# pocket = np.array([pocket_x, pocket_y, pocket_z]).T

# distance_list = []
# for atom in pocket:
#     distance_list.append(np.linalg.norm(atom - ligand_center))
# print(f"Bounding Box [{protein_name}] : {np.round(np.mean(distance_list), 3)} A")

Bounding Box [PDGFRb_20] : 16.728 A


In [4]:
import torch
import torch_scatter
import numpy as np
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
class ProteinLigandData(Data):

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    @staticmethod
    def from_protein_ligand_dicts(protein_dict=None, ligand_dict=None, residue_dict=None, inter_dict=None, **kwargs):
        instance = ProteinLigandData(**kwargs)

        if protein_dict is not None:
            for key, item in protein_dict.items():
                instance['protein_' + key] = item

        if ligand_dict is not None:
            for key, item in ligand_dict.items():
                instance['ligand_' + key] = item
                
        if residue_dict is not None:
            for key, item in residue_dict.items():
                instance[key] = item
                
        if inter_dict is not None:
            for key, item in inter_dict.items():
                instance[key] = item
        
        instance['ligand_nbh_list'] = {i.item(): [j.item() for k, j in enumerate(instance.ligand_bond_index[1])
                                                  if instance.ligand_bond_index[0, k].item() == i]
                                       for i in instance.ligand_bond_index[0]}
        
        """
        Here it for "residue_pos"
        """
        instance['residue_pos'] = torch.stack([instance['pos_CA'], 
                                               instance['pos_C'], 
                                               instance['pos_N'], 
                                               instance['pos_O']], dim=1)
        
        
        return instance

    def __inc__(self, key, value, *args, **kwargs):
        if key == 'ligand_bond_index':
            return self['ligand_element'].size(0)
        else:
            return super().__inc__(key, value)
def torchify_dict(data):
    output = {}
    for k, v in data.items():
        if isinstance(v, np.ndarray):
            output[k] = torch.from_numpy(v)
        else:
            output[k] = v
    return output

In [9]:
interactions = get_interaction(complex_path, sdf_path)
pdb_block, sdf_block = load_item(sdf_path, pocket_path)
pocket_dict = PDBProtein(pdb_block).to_dict_atom()
residue_dict = PDBProtein(pdb_block).to_dict_residue()
ligand_dict = parse_sdf_file(sdf_path)
data = ProteinLigandData.from_protein_ligand_dicts(
    protein_dict=torchify_dict(pocket_dict),
    ligand_dict=torchify_dict(ligand_dict),
    residue_dict=torchify_dict(residue_dict),
)
data.protein_filename = pdb_path
data.ligand_filename = sdf_path
#data = data.to_dict()

In [ ]:
device = 'cuda:0'
config_path = '/home/csy/work/3D/PIDiff_PINN/scripts/real_world_validation/sampling.yml'
sample_size = 10
config = misc.load_config(config_path)
misc.seed_all(config.sample.seed)
ckpt = torch.load(config.model.checkpoint, map_location=device)
protein_featurizer = trans.FeaturizeProteinAtom()
ligand_atom_mode = ckpt['config'].data.transform.ligand_atom_mode
ligand_featurizer = trans.FeaturizeLigandAtom(ligand_atom_mode)
transform = Compose([
    protein_featurizer,
    ligand_featurizer,
    trans.FeaturizeLigandBond(),
])
model = ScorePosNet3D(
    ckpt['config'].model,
    protein_atom_feature_dim=protein_featurizer.feature_dim,
    ligand_atom_feature_dim=ligand_featurizer.feature_dim
).to(device)
model.load_state_dict(ckpt['model'])
data = transform(data)

all_pred_pos, all_pred_v, pred_pos_traj, pred_v_traj, pred_v0_traj, pred_vt_traj, time_list = sample_diffusion_ligand(
    model, data, config.sample.num_samples,
    batch_size=sample_size, device=device,
    num_steps=config.sample.num_steps,
    pos_only=config.sample.pos_only,
    center_pos_mode=config.sample.center_pos_mode,
    sample_num_atoms=config.sample.sample_num_atoms
)

In [16]:
result = {
    'data': data,
    'pred_ligand_pos': all_pred_pos,
    'pred_ligand_v': all_pred_v,
    'pred_ligand_pos_traj': pred_pos_traj,
    'pred_ligand_v_traj': pred_v_traj
}

In [18]:
protein_path = f'/home/csy/work/3D/PIDiff_PINN/scripts/real_world_validation/Kinase/{protein_name}/'
result_path = os.path.join(protein_path, 'result')
os.makedirs(result_path, exist_ok=True)

In [19]:
with open(result_path + "/mol_result.pickle",'wb') as fw:
    pickle.dump(result, fw)

In [ ]:
with open(result_path + "/mol_result.pickle", 'rb') as fr:
    user_loaded = pickle.load(fr)
    


In [10]:
# result_path = sdf_path.replace('.sdf', '_result.pt')
# torch.save(result, result_path)

In [5]:
protein_list = ['4G5J_EGFR', '5AAA_ALK', '5MQT_KIT','3DKF_MET']
for protein_name in tqdm(protein_list):
    sdf_path = f'/home/csy/work/3D/PIDiff/scripts/real_world_validation/Kinase/{protein_name}/ligand.sdf'
    pdb_path = f'/home/csy/work/3D/PIDiff/scripts/real_world_validation/Kinase/{protein_name}/receptor.pdb'
    pocket_path = f'/home/csy/work/3D/PIDiff/scripts/real_world_validation/Kinase/{protein_name}/'
    pocket_path = process_item(sdf_path, pdb_path, pocket_path)
    complex_path = sdf2pdb(sdf_path, pocket_path) 
    pocket_path = f'/home/csy/work/3D/PIDiff/scripts/real_world_validation/Kinase/{protein_name}/receptor_pocket.pdb'
    complex_path = f'/home/csy/work/3D/PIDiff/scripts/real_world_validation/Kinase/{protein_name}/receptor_pocketcomplex.pdb'
    
    interactions = get_interaction(complex_path, sdf_path)
    pdb_block, sdf_block = load_item(sdf_path, pocket_path)
    pocket_dict = PDBProtein(pdb_block).to_dict_atom()
    residue_dict = PDBProtein(pdb_block).to_dict_residue()
    ligand_dict = parse_sdf_file(sdf_path)
    interactions_dict = {"IntramolInteraction":interactions}
    data = ProteinLigandData.from_protein_ligand_dicts(
        protein_dict=torchify_dict(pocket_dict),
        ligand_dict=torchify_dict(ligand_dict),
        residue_dict=torchify_dict(residue_dict),
        inter_dict=torchify_dict(interactions_dict),
    )
    data.protein_filename = pdb_path
    data.ligand_filename = sdf_path

    device = 'cuda:0'
    config_path = '/home/csy/work/3D/PIDiff/scripts/real_world_validation/sampling.yml'
    sample_size = 10
    config = misc.load_config(config_path)
    misc.seed_all(config.sample.seed)
    ckpt = torch.load(config.model.checkpoint, map_location=device)
    protein_featurizer = trans.FeaturizeProteinAtom()
    ligand_atom_mode = ckpt['config'].data.transform.ligand_atom_mode
    ligand_featurizer = trans.FeaturizeLigandAtom(ligand_atom_mode)
    transform = Compose([
        protein_featurizer,
        ligand_featurizer,
        trans.FeaturizeLigandBond(),
    ])
    model = ScorePosNet3D(
        ckpt['config'].model,
        protein_atom_feature_dim=protein_featurizer.feature_dim,
        ligand_atom_feature_dim=ligand_featurizer.feature_dim
    ).to(device)
    model.load_state_dict(ckpt['model'])
    data = transform(data)

    all_pred_pos, all_pred_v, pred_pos_traj, pred_v_traj, pred_v0_traj, pred_vt_traj, time_list = sample_diffusion_ligand(
        model, data, config.sample.num_samples,
        batch_size=sample_size, device=device,
        num_steps=config.sample.num_steps,
        pos_only=config.sample.pos_only,
        center_pos_mode=config.sample.center_pos_mode,
        sample_num_atoms=config.sample.sample_num_atoms
    )

    result = {
        'data': data,
        'pred_ligand_pos': all_pred_pos,
        'pred_ligand_v': all_pred_v,
        'pred_ligand_pos_traj': pred_pos_traj,
        'pred_ligand_v_traj': pred_v_traj
    }

    protein_path = f'/home/csy/work/3D/PIDiff/scripts/real_world_validation/Kinase/{protein_name}/'
    result_path = os.path.join(protein_path, 'result')
    os.makedirs(result_path, exist_ok=True)

    with open(result_path + "/mol_result.pickle",'wb') as fw:
        pickle.dump(result, fw)


  0%|          | 0/4 [00:00<?, ?it/s]

CompletedProcess(args=['obabel', '-isdf', '/home/csy/work/3D/targetdiff_PINN/scripts/real_world_validation/Kinase/4G5J_EGFR/ligand.sdf', '-opdb', '-O', '/home/csy/work/3D/targetdiff_PINN/scripts/real_world_validation/Kinase/4G5J_EGFR/ligand.pdb'], returncode=0, stdout='', stderr='1 molecule converted\n')


  0%|          | 0/100 [00:00<?, ?it/s]

CompletedProcess(args=['obabel', '-isdf', '/home/csy/work/3D/targetdiff_PINN/scripts/real_world_validation/Kinase/5AAA_ALK/ligand.sdf', '-opdb', '-O', '/home/csy/work/3D/targetdiff_PINN/scripts/real_world_validation/Kinase/5AAA_ALK/ligand.pdb'], returncode=0, stdout='', stderr='1 molecule converted\n')


  0%|          | 0/100 [00:00<?, ?it/s]

CompletedProcess(args=['obabel', '-isdf', '/home/csy/work/3D/targetdiff_PINN/scripts/real_world_validation/Kinase/5MQT_KIT/ligand.sdf', '-opdb', '-O', '/home/csy/work/3D/targetdiff_PINN/scripts/real_world_validation/Kinase/5MQT_KIT/ligand.pdb'], returncode=0, stdout='', stderr='1 molecule converted\n')


  0%|          | 0/100 [00:00<?, ?it/s]

CompletedProcess(args=['obabel', '-isdf', '/home/csy/work/3D/targetdiff_PINN/scripts/real_world_validation/Kinase/3DKF_MET/ligand.sdf', '-opdb', '-O', '/home/csy/work/3D/targetdiff_PINN/scripts/real_world_validation/Kinase/3DKF_MET/ligand.pdb'], returncode=0, stdout='', stderr='1 molecule converted\n')


  0%|          | 0/100 [00:00<?, ?it/s]